<h1>Recommendation Model<h1>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import joblib

In [2]:
df = pd.read_csv("student_recommendation_dataset.csv")
new_features_df = pd.read_csv("new_features_rwanda_students.csv", dtype=str)

In [3]:
df = pd.concat([df, new_features_df], axis=1) #merge both dfs

df.drop(columns=["student_id"], inplace=True)
df.head()

age  gender school_type location  math_score  english_score  science_score  \
0   18  Female      Public    Rural          89             78             99   
1   19  Female     Private    Rural          84             60             52   
2   17  Female      Public    Rural          41             49             54   
3   19    Male      Public    Rural          80             77             57   
4   19  Female      Public    Urban          60             85             90   

   history_score  attendance_rate  study_hours_per_week  ...  \
0             73         0.959424                    22  ...   
1             50         0.913279                    26  ...   
2             75         0.912150                    26  ...   
3             90         0.783198                     5  ...   
4             56         0.759202                    21  ...   

   job_demand_business  job_demand_healthcare  job_demand_arts  \
0             0.607863               0.283746         0.630851   
1             0.403364               0.436582         0.349585   
2             0.676472               0.633797         0.292671   
3             0.189933               0.450052         0.392588   
4             0.818747               0.337817         0.342023   

   job_demand_humanities  parental_education_level  household_income  \
0               0.433464                  Tertiary              1083   
1               0.396728                 Secondary               803   
2               0.938054                 Secondary               575   
3               0.153135                   Primary               691   
4               0.009847                       NaN               115   

   internet_access  recommended_stream  parental_career  \
0             True                STEM       Healthcare   
1             True          Healthcare             Arts   
2            False          Humanities         Business   
3             True          Healthcare             Arts   
4            False            Business             Arts   

   extracurricular_activity  
0                       NaN  
1     Entrepreneurship Club  
2     Entrepreneurship Club  
3                     Music  
4              Science Club  

[5 rows x 26 columns]

In [4]:
df.fillna("None", inplace=True)

df = pd.get_dummies(df, columns=["gender", "school_type", "location", "parental_education_level", "internet_access", "parental_career", "extracurricular_activity"], drop_first=True)

- Normalize numerical features

In [5]:
scaler = StandardScaler()
numerical_cols = ["math_score", "english_score", "science_score", "history_score", 
                  "attendance_rate", "study_hours_per_week", "household_income"]

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


- Compute similarity scores based on student profiles

In [6]:
similarity_matrix = cosine_similarity(df[numerical_cols])


- Function to recommend top-3 streams based on similarity

In [7]:
def recommend_streams(student_index, df, similarity_matrix, top_n=3):
    scores = similarity_matrix[student_index]
    sorted_indices = np.argsort(scores)[::-1][1:top_n+1]  # Exclude self
    
    return df.iloc[sorted_indices]["recommended_stream"].tolist()

#sample to get recommendations for student
recommend_streams(0, df, similarity_matrix)

['STEM', 'Arts', 'Humanities']

- Train a k-NN model for collaborative filtering

In [8]:
knn_model = NearestNeighbors(n_neighbors=5, metric="cosine")
knn_model.fit(df[numerical_cols])

NearestNeighbors(metric='cosine')

-  Function to recommend streams using hybrid approach

In [9]:

def hybrid_recommend(student_index, df, knn_model, similarity_matrix, top_n=3):
    # Get content-based recommendations
    content_recs = recommend_streams(student_index, df, similarity_matrix, top_n)
    
    # Get collaborative filtering recommendations (nearest neighbors)
    distances, indices = knn_model.kneighbors([df.iloc[student_index][numerical_cols]], n_neighbors=top_n+1)
    collab_recs = df.iloc[indices[0][1:]]["recommended_stream"].tolist()  # Exclude self
    
    # Combine both recommendations
    final_recommendations = list(set(content_recs + collab_recs))[:top_n]
    return final_recommendations

# Example: Hybrid recommendations for student at index 0
hybrid_recommend(2, df, knn_model, similarity_matrix)


c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Healthcare', 'STEM', 'Arts']

- save the models

In [10]:
joblib.dump(knn_model, "knn_recommender_model.pkl") #k-NN model

joblib.dump(similarity_matrix, "similarity_matrix.pkl") #similarity matrix

df.to_csv("processed_student_data.csv", index=False) #processed dataset